In [20]:
import numpy as np
from PIL import Image
from scipy import ndimage
from helper_v1 import *

In [76]:
def initialize_params(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    np.random.seed(3)
    print('dim ',dim)
    b = 0
    w = np.random.randn(dim, 1) * 0.01
#     w = np.zeros([dim, 1])

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [72]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.

    """
    
    costs = []
    grads = {}
    cost = None
    m = X.shape[1]
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        
        # activation function is sigmoid
        z = np.dot(w.T, X) + b
        A, cache = sigmoid(z)
        # dZ = A-Y         
        grads["dw"] = (1 / m) * np.dot(X, (A - Y).T)
        grads["db"] = (1 / m) * np.sum(A - Y)
        cost = - (1 / m) * np.sum((Y * np.log(A) + (1 - Y) * np.log(1 - A)))
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
            print_cost = True
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [56]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = 1 / (1 + np.exp(-(np.dot(w.T, X) + b)))
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        Y_prediction = np.where(A > 0.5, 1, 0)
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction    

In [24]:
def train_model(train_set_x, train_set_y):
    w, b = initialize_params(train_set_x.shape[0])
    params, grads, costs = optimize(w, b, train_set_x_flatten, train_set_y, num_iterations = 1500, learning_rate=0.01, print_cost = False)
    return params, grads, costs

In [70]:
# image = np.array(ndimage.imread(fname, flatten=False))
# my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
# my_image = my_image/255

train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_data()
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T
train_set_x_flatten=train_set_x_flatten/255
test_set_x_flatten=test_set_x_flatten/255
print(train_set_x_flatten.shape)

(12288, 209)


In [77]:
params, grads, costs = train_model(test_set_x_flatten, train_set_y_orig)

dim  12288
Cost after iteration 0: 0.690909
Cost after iteration 100: 1.001207
Cost after iteration 200: 0.609737
Cost after iteration 300: 1.019847
Cost after iteration 400: 0.730757
Cost after iteration 500: 0.463708
Cost after iteration 600: 0.249762
Cost after iteration 700: 0.154532
Cost after iteration 800: 0.134770
Cost after iteration 900: 0.124428
Cost after iteration 1000: 0.115966
Cost after iteration 1100: 0.108713
Cost after iteration 1200: 0.102356
Cost after iteration 1300: 0.096709
Cost after iteration 1400: 0.091649


In [78]:
train_predictions = predict(params['w'], params['b'], train_set_x_flatten)

In [79]:
print("train accuracy: {} %".format(100 - np.mean(np.abs(train_predictions - train_set_y_orig)) * 100))

train accuracy: 99.52153110047847 %


In [80]:
test_predictions = predict(params['w'], params['b'], test_set_x_flatten)

In [81]:
print("test accuracy: {} %".format(100 - np.mean(np.abs(test_predictions - test_set_y_orig)) * 100))

test accuracy: 66.0 %
